In [ ]:
# 04_transfer_learning.ipynb - Cellule 1
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report

import sys
sys.path.append('..')
from src.visualization import plot_training_history, plot_confusion_matrix
from src.evaluate import evaluate_model

In [ ]:
# Cellule 2 (identique au notebook baseline)
train_dir = '../data/train'
val_dir = '../data/val'
test_dir = '../data/test'

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
NUM_CLASSES = 38
EPOCHS = 20

In [ ]:
# Cellule 3
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)

In [ ]:
# Cellule 4
# Charger la base de VGG16 sans la tête de classification
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))

# Geler les poids de la base
base_model.trainable = False

# Ajouter la tête personnalisée
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=outputs)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Cellule 5
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('../models/vgg16_feature_extraction.h5', save_best_only=True)
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
# Cellule 6
# Dégeler les dernières couches du modèle de base
base_model.trainable = True
for layer in base_model.layers[:15]:  # geler les 15 premières couches
    layer.trainable = False

# Recompiler avec un learning rate plus faible
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continuer l'entraînement
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # quelques époques supplémentaires
    callbacks=callbacks
)

In [ ]:
# Cellule 7
plot_training_history(history)
plot_training_history(history_fine)  # si vous voulez les deux séparément

In [ ]:
# Cellule 8
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
# Cellule 9
test_generator.reset()
predictions = model.predict(test_generator)
pred_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

cm = confusion_matrix(true_classes, pred_classes)
plot_confusion_matrix(cm, class_labels, title='Confusion Matrix - Transfer Learning')

print(classification_report(true_classes, pred_classes, target_names=class_labels))

In [ ]:
# Cellule 10
model.save('../models/transfer_learning_final.h5')